# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    #print(df_eng.columns)
    #TODO 1: Add your code here
    df_eng["Original Language"] = "English"
    #df_eng.rename(columns= {'Movie / TV Series' : 'Title'})
    df_eng.rename(columns= {'Movie / TV Series' : 'Title'}, inplace = 'True')
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr["Original Language"] = "French"
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp["Original Language"] = "Spanish"
    print(df_eng.columns)
    df_fr.columns = df_eng.columns
    df_sp.columns = df_eng.columns
    #df = pd.concat([df_eng, df_fr, df_sp])
    df = pd.concat( [df_eng, df_fr, df_sp] , ignore_index = True )
    df.to_csv("data/movie_reviews_df.csv")
    
    return df

df = preprocess_data()

Index(['Title', 'Year', 'Synopsis', 'Review', 'Original Language'], dtype='object')


In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
14,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",French
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English
11,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",French
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English
28,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",Spanish
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
23,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin...",Spanish
25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",Spanish


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors = "pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return decoded

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df[ df["Original Language"] == "French"]["Review"]
#print(fr_reviews)
#fr_reviews_en = translate(fr_reviews, model = fr_en_model, tokenizer = fr_en_tokenizer)
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html - Source for apply in place of a for loop
fr_reviews_en = fr_reviews.apply(lambda x: translate(x, model = fr_en_model, tokenizer = fr_en_tokenizer))
#print(fr_reviews_en)

# filter synopsis in French and translate to English
fr_synopsis = df[ df["Original Language"] == "French"]["Synopsis"]
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, model = fr_en_model, tokenizer = fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df[ df["Original Language"] == "Spanish"]["Review"]
es_reviews_en = es_reviews.apply(lambda x: translate(x, model = es_en_model, tokenizer = es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df[ df["Original Language"] == "Spanish"]["Synopsis"]
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, model = es_en_model, tokenizer = es_en_tokenizer))

# update dataframe with translated text
# add the translated french reviews, overwrite the existing data
df.loc[ df["Original Language"] == "French",'Review'] = fr_reviews_en

df.sample(10)

/opt/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


,Title,Year,Synopsis,Review,Original Language
12,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie is an absolutely charming film that wi...",French
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English
15,Le Dîner de Cons,1998,Le film suit l'histoire d'un groupe d'amis ric...,"""I didn't like this movie at all. The concept ...",French
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
16,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,"""I can't believe I've wasted time watching thi...",French
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
21,La Casa de Papel,(2017-2021),Esta serie de televisión española sigue a un g...,"""La Casa de Papel es una serie emocionante y a...",Spanish
24,Amores perros,2000,Tres historias se entrelazan en esta película ...,"""Amores perros es una película intensa y conmo...",Spanish
17,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""This film is a complete disappointment. The j...",French


### Adding other translated values in
Now that we can confirm that the translated reviews can be seen as replaced for the french reviews, we can do the same for **french synopsis and all the spanish translated items**
.loc function was used based on this webpage: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html


In [6]:
#Replace all othr cells with translated values 
df.loc[ df["Original Language"] == "French",'Synopsis'] = fr_synopsis_en
df.loc[ df["Original Language"] == "Spanish",'Review'] = es_reviews_en
df.loc[ df["Original Language"] == "Spanish",'Synopsis'] = es_synopsis_en
df.to_csv("data/movie_reviews_translated.csv")

In [7]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",French
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,"""Rome is a beautiful and moving film that pays...",Spanish
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",Spanish
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",English
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",Spanish
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [89]:
# TODO 5: Update the code below
# load sentiment analysis model
#model_name = "distilbert-base-uncased"
#sentiment_classifier = pipeline("sentiment-analysis", model = model_name)
#label_map = {"LABEL_1":"NEGATIVE", "LABEL_2":"POSITIVE"}
# TODO 6: Complete the function below
#def analyze_sentiment(text, classifier, label_map):
#    """
#    function to perform sentiment analysis on a text using a model
#    """
#    sentiment_result = classifier(text)
#    print(sentiment_result)
#    sentiment_label = sentiment_result[0]['label']
#    print(sentiment_label)
#    sentiment_category = label_map.get(sentiment_label, "UNKNOWN")
#    print(sentiment_category)
    
#    return sentiment_category

#This was always returning Uknown/ Negative. Something was not correct. Moved to a different model instead

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [17]:
# TODO 5: Update the code below
# load sentiment analysis model
#went with distilbert-base-uncased-finetuned-sst-2-english
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
def analyze_sentiment(text):
    
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    predicted_class_id = logits.argmax().item()
    print(model.config.id2label[predicted_class_id])
    return model.config.id2label[predicted_class_id]

In [18]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df['Sentiment'] = df['Review'].apply( lambda x: analyze_sentiment(x) )

POSITIVE
POSITIVE
POSITIVE
POSITIVE
POSITIVE
NEGATIVE
NEGATIVE
NEGATIVE
NEGATIVE
NEGATIVE
POSITIVE
POSITIVE
POSITIVE
POSITIVE
POSITIVE
NEGATIVE
NEGATIVE
NEGATIVE
NEGATIVE
NEGATIVE
POSITIVE
POSITIVE
POSITIVE
POSITIVE
POSITIVE
NEGATIVE
NEGATIVE
NEGATIVE
NEGATIVE
NEGATIVE


In [19]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
27,El Bar,2017,A group of people are trapped in a bar after M...,"""The Bar is a ridiculous and meaningless film ...",Spanish,NEGATIVE
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English,NEGATIVE
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English,NEGATIVE
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",Spanish,NEGATIVE
11,Intouchables,2011,This film tells the story of the unlikely frie...,"""Untouchables is an incredibly touching film w...",French,POSITIVE
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",French,NEGATIVE
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French,POSITIVE
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French,POSITIVE
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English,NEGATIVE
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",Spanish,NEGATIVE


In [20]:
# export the results to a .csv file
df.to_csv("data/reviews_with_sentiment.csv", index=False)